In [40]:
import os
import json
import re
import numpy as np
import pandas as pd

In [31]:
def get_files_sizes_features(files_sizes, all_files_sizes):
    size_features = []
    size_features.append(min(files_sizes))
    size_features.append(max(files_sizes))
    size_features.append(sum(files_sizes)/len(files_sizes))
    size_features.append(len(files_sizes))
    size_features.append(sum(files_sizes))
                                               
    return size_features

In [29]:
def find_file_size_s3(names, output):
    file_sizes = []
    for name in names:
        file_size = re.search('([0-9]+) ' + name, output)
        file_sizes.append(file_size.group(1))
    return file_sizes

In [59]:
def get_inout_files_features(s3_path, files):
    global ls_result1
    files_sizes = []
    for file in files:
        file_size = re.search('([0-9]+) ' + file['name'], ls_result1)
        files_sizes.append(file_size.group(1))
    files_sizes = list(map(int, files_sizes))
    all_sizes = re.findall('([0-9]+) \S*', ls_result1)
    np_sizes = np.array(list(map(int, all_sizes)))
    return get_files_sizes_features(files_sizes, np_sizes)

In [45]:
def get_task_count_features(task_name, task_map):
    features = []
    features.append(task_map[task_name])
    sum_task_count = 0
    for task in task_map.keys():
        sum_task_count += task_map[task]
    features.append(sum_task_count)

    features.append(list(task_map.keys()).index(task_name))
    
    return features

In [6]:
def get_tasks_count_map(workflow_file_path):
    with open(workflow_file_path) as json_workflow:
        process_number_map = {}
        data = json.load(json_workflow)
        for process in data['processes']:
            process_name = process['name']
            process_number_map = append_proccess_to_map(process_number_map, process_name)
        return process_number_map

In [7]:
def ls_aws_s3_and_capture(s3_path):
    result = ! aws s3 ls {s3_path}
    return ' '.join(result)

In [60]:
def get_compiles_files_features(s3_path, task_name):
    global compiles_ls_result1
    all_sizes = re.findall('([0-9]+) \S*', compiles_ls_result1)
    np_compiled_sizes = np.array(list(map(int, all_sizes)))
    file_size = int(find_file_size_s3([task_name], compiles_ls_result1)[0])
    return file_size

In [18]:
def append_proccess_to_map(process_number_map, process):
    if(process in process_number_map):
        process_number_map[process] += 1
    else:
        process_number_map[process] = 1
    return process_number_map

In [9]:
def get_all_features(task_name, ins, outs, task_count_map):
    all_features = []
    all_features.extend(get_task_count_features(task_name, task_count_map))
    all_features.extend(get_inout_files_features('cegielskir/montageV2_6-0.35/', ins))
    all_features.extend(get_inout_files_features('cegielskir/montageV2_6-0.35/', outs))
    all_features.extend([(get_compiles_files_features('cegielskir/montageV2_6-compiles/', task_name))])
    return all_features   

In [68]:
task_count_map = get_tasks_count_map('./dc-workflow-0.15.json')
global ls_result1, ls_result_compiles1
ls_result1 = ls_aws_s3_and_capture('cegielskir/dc-start-montage-0_15/')
compiles_ls_result1 = ls_aws_s3_and_capture('cegielskir/montageV2_6-compiles/')
features = []
for filename in os.listdir('./montage-0_15/'):
    if '.json' in filename:
        with open('./montage-0_15/' + filename) as single_workflow:
            task_id = filename[:-5]
            data = json.load(single_workflow)
            task_name = data['processes'][0]['name']
            process = data['processes'][0]
            config = process['config']
            ins = [ data['signals'][i] for i in process['ins'] ]
            outs = [ data['signals'][i] for i in process['outs'] ]
            core_features = get_all_features(task_name, ins, outs, task_count_map)
            core_features.insert(0, task_id)
            features.append(core_features)

In [69]:
features

[['mDiffFit_8',
  198,
  310,
  1,
  277,
  4152960,
  3318967.4,
  5,
  16594837,
  263,
  263,
  263.0,
  1,
  263,
  746640],
 ['mProject_30',
  48,
  310,
  0,
  277,
  1469047,
  734662.0,
  2,
  1469324,
  4161600,
  4161600,
  4161600.0,
  2,
  8323200,
  5930728],
 ['mBackground_20',
  48,
  310,
  4,
  936,
  4161600,
  2081755.0,
  4,
  8327020,
  4161600,
  4161600,
  4161600.0,
  2,
  8323200,
  5793368],
 ['mDiffFit_59',
  198,
  310,
  1,
  277,
  4152960,
  3322423.4,
  5,
  16612117,
  266,
  266,
  266.0,
  1,
  266,
  746640],
 ['mBackground_15',
  48,
  310,
  4,
  936,
  4144320,
  2073115.0,
  4,
  8292460,
  4144320,
  4144320,
  4144320.0,
  2,
  8288640,
  5793368],
 ['mDiffFit_126',
  198,
  310,
  1,
  277,
  4161600,
  3329335.4,
  5,
  16646677,
  267,
  267,
  267.0,
  1,
  267,
  746640],
 ['mProject_44',
  48,
  310,
  0,
  277,
  1471563,
  735920.0,
  2,
  1471840,
  4144320,
  4144320,
  4144320.0,
  2,
  8288640,
  5930728],
 ['mProject_31',
  48,
  3

In [70]:
d = pd.DataFrame(features, columns=['task_id','task_count', 'all_tasks_count', 'task_type', 'min_size_in', 'max_size_in', 'avg_size_in', 'count_in', 'sum_in', 'min_size_out', 'max_size_out', 'avg_size_out', 'count_out', 'sum_out', 'exec_file_size'])


In [71]:
d

,task_id,task_count,all_tasks_count,task_type,min_size_in,max_size_in,avg_size_in,count_in,sum_in,min_size_out,max_size_out,avg_size_out,count_out,sum_out,exec_file_size
0,mDiffFit_8,198,310,1,277,4152960,3318967.4,5,16594837,263,263,263.0,1,263,746640
1,mProject_30,48,310,0,277,1469047,734662.0,2,1469324,4161600,4161600,4161600.0,2,8323200,5930728
2,mBackground_20,48,310,4,936,4161600,2081755.0,4,8327020,4161600,4161600,4161600.0,2,8323200,5793368
3,mDiffFit_59,198,310,1,277,4152960,3322423.4,5,16612117,266,266,266.0,1,266,746640
4,mBackground_15,48,310,4,936,4144320,2073115.0,4,8292460,4144320,4144320,4144320.0,2,8288640,5793368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,mDiffFit_184,198,310,1,277,4144320,3315511.4,5,16577557,270,270,270.0,1,270,746640
306,mBackground_19,48,310,4,936,4161600,2081755.0,4,8327020,4161600,4161600,4161600.0,2,8323200,5793368
307,mDiffFit_136,198,310,1,277,4144320,3312055.4,5,16560277,268,268,268.0,1,268,746640
308,mProject_26,48,310,0,277,1473216,736746.5,2,1473493,4161600,4161600,4161600.0,2,8323200,5930728


In [72]:
d.to_csv ('./montage-0.15-core.csv', index = False, header=True)
